## Basic Linear Regression
------------------------------
The following examples (tutorials) are from TensorFlow official website [here](https://www.tensorflow.org/get_started/get_started).

### Basic Linear Regression (low-level)
------------------------------

> References (ordered by the following code):
> * [Variables](https://www.tensorflow.org/api_docs/python/tf/Variable)
> * [PlaceHolder](https://www.tensorflow.org/api_docs/python/tf/placeholder)
> * [tf.reduce_sum](https://www.tensorflow.org/api_docs/python/tf/reduce_sum)
> * [tf.train.GradientDescentOptimizer](https://www.tensorflow.org/api_docs/python/tf/train/GradientDescentOptimizer)
> * [Optimizers](https://www.tensorflow.org/versions/master/api_docs/python/train/)
> * [tf.train.Optimizer.minimize](https://www.tensorflow.org/versions/master/api_docs/python/train/optimizers#Optimizer.minimize)
> * [tf.global_variables_initializer](https://www.tensorflow.org/api_docs/python/tf/global_variables_initializer)
> * [tf.Session.run](https://www.tensorflow.org/api_docs/python/tf/Session#run)

In [1]:
import numpy as np
import tensorflow as tf


# Model parameters
# -------------------------------------------------------
# Variables
# __init__(initial_value=None, trainable=True, collections=None,
#             validate_shape=True, caching_device=None, name=None,
#             variable_def=None, dtype=None, expected_shape=None, import_scope=None)
# shortcut version
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)

# formal version
# W = tf.Variable(initial_value=[.3], dtype=tf.float32)
# b = tf.Variable(initial_value=[-.3], dtype=tf.float32)


# Model input and output
# -------------------------------------------------------
# PlaceHolder
# __init__(initial_value=None, name=None, dtype=None)
# shortcut version
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# formal version
# x = tf.placeholder(name='x', dtype=tf.float32)
# y = tf.placeholder(name='y', dtype=tf.float32)

linear_model = W * x + b


# Loss
# -------------------------------------------------------
# tf.reduce_sum(input_tensor, axis=None, keep_dims=False, name=None, reduction_indices=None)
# shortcut version
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares

# formal version
# loss = tf.reduce_sum(tf.square(linear_model - y), name='loss') # sum of the squares


# Optimizer
# -------------------------------------------------------
# tf.train.GradientDescentOptimizer
# __init__(learning_rate, use_locking=False, name='GradientDescent')
# shortcut version
optimizer = tf.train.GradientDescentOptimizer(0.01)

# formal version
# optimizer = tf.train.GradientDescentOptimizer(0.01, name='GradientDescent')

# tf.train.Optimizer.minimize(loss, global_step=None, var_list=None, gate_gradients=1,
#                                 aggregation_method=None, colocate_gradients_with_ops=False,
#                                 name=None, grad_loss=None)
# shortcut version
train = optimizer.minimize(loss)

# formal version
# train = optimizer.minimize(loss, name='minimize')


# Training Data
# -------------------------------------------------------
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]


# Training Loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong


# tf.Session.run(fetches, feed_dict=None, options=None, run_metadata=None)
for i in range(1000):
    sess.run(train, {x:x_train, y:y_train})


# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


### Basic Linear Regression (by tf.contrib.learn - high level)
------------------------------

Here's TensorFlow's introduction:
> tf.contrib.learn is a high-level TensorFlow library that simplifies the mechanics of machine learning, including the following:
> * running training loops
> * running evaluation loops
> * managing data sets
> * managing feeding  
> 
> Also, tf.contrib.learn defines many common models.
> 
> References (ordered by the following code):
> * [tf.contrib.layers.real_valued_column](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/real_valued_column)
> * [tf.contrib.learn.LinearRegressor](https://www.tensorflow.org/api_docs/python/tf/contrib/learn/LinearRegressor)
> * [tf.contrib.learn.LinearRegressor.fit](https://www.tensorflow.org/api_docs/python/tf/contrib/learn/LinearRegressor#fit)
> * [tf.contrib.learn.LinearRegressor.evaluate](https://www.tensorflow.org/api_docs/python/tf/contrib/learn/LinearRegressor#evaluate)

In [2]:
import tensorflow as tf
import numpy as np


# Declare list of features. We only have one real-valued feature.
# There are many other types of columns that are more complicated and useful.
# ----------------------------------------------------------------------------
# tf.contrib.layers.real_valued_column(column_name, dimension=1,
#                                         default_value=None, dtype=tf.float32,normalizer=None)
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]


# An estimator is the front end to invoke training (fitting) and evaluation (inference).
# There are many predefined types like linear regression, logistic regression, 
# linear classification, logistic classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
# ----------------------------------------------------------------------------
# __init__(feature_columns, model_dir=None, weight_column_name=None, optimizer=None,
#             gradient_clip_norm=None, enable_centered_bias=False, label_dimension=1,
#             _joint_weights=False, config=None, feature_engineering_fn=None)
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)


# TensorFlow provides many helper methods to read and set up data sets.
# Here we use `numpy_input_fn`. We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4,
                                              num_epochs=1000)

# We can invoke 1000 training steps by invoking the `fit` method and passing the
# training data set.
estimator.fit(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did. In a real example, we would want
# to use a separate validation and testing data set to avoid overfitting.
estimator.evaluate(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10c3fa6d8>, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_is_chief': True, '_task_id': 0, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_task_type': None, '_environment': 'local', '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/cq/zskyb43

{'global_step': 1000, 'loss': 2.2615934e-09}

### Basic Linear Regression (customized by tf.contrib.learn - high level)
------------------------------

> References (ordered by the following code):
> * [Creating Estimators in tf.contrib.learn](https://www.tensorflow.org/versions/r0.12/tutorials/estimators/)

In [3]:
import numpy as np
import tensorflow as tf


# Declare list of features, we only have one real-valued feature
def model(features, labels, mode):
    # Build a linear model and predict values
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    
    # Loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                    tf.assign_add(global_step, 1))
    
    # ModelFnOps connects subgraphs we built to the
    # appropriate functionality.
    return tf.contrib.learn.ModelFnOps(mode=mode, predictions=y, loss= loss, train_op=train)

estimator = tf.contrib.learn.Estimator(model_fn=model)

# define our data set
x=np.array([1., 2., 3., 4.])
y=np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x}, y, 4, num_epochs=1000)

# train
estimator.fit(input_fn=input_fn, steps=1000)

# evaluate our model
print(estimator.evaluate(input_fn=input_fn, steps=10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1103c4198>, '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_is_chief': True, '_task_id': 0, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_task_type': None, '_environment': 'local', '_master': ''}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/cq/zskyb43j2tx3c0bhsb4s6q540000gn/T/tmpi3sb5ekq/model.ckpt.
INFO:tensorflow:step = 1, loss = 13.9438591404
INFO:tensorflow:global_step/sec: 1582.25
INFO:tensorflow:step = 101, loss = 0.0237522604638
INFO:tensorflow:global_step/sec: 1638.3
INFO:tensorflow:step = 201, loss = 0.00311036098364
INFO:tensorflow:global_step/sec: 1607.92
INFO:te